In [ ]:
import requests
import pandas as pd
from datetime import date

In [ ]:
def url_compare(url, comp):
    #Get data
    d = requests.get("https://apiv2.ahrefs.com?token=" + token + "&from=domain_rating"
                     + "&output=json&mode=prefix&select=domain_rating&target=" + url)
    u = requests.get("https://apiv2.ahrefs.com?token=" + token + "&from=pages"
                     + "&output=json&mode=exact&select=ahrefs_rank&target=" + url)
    cd = requests.get("https://apiv2.ahrefs.com?token=" + token + "&from=domain_rating"
                     + "&output=json&mode=prefix&select=domain_rating&target=" + comp)
    cu = requests.get("https://apiv2.ahrefs.com?token=" + token + "&from=pages"
                     + "&output=json&mode=exact&select=ahrefs_rank&target=" + comp)

    #Convert to json
    domain_json = d.json()
    url_json = u.json()
    cdomain_json = cd.json()
    curl_json = cu.json()

    #Build DF
    urls = {'URL': [url[23:]]}
    df = pd.DataFrame(urls, columns = ['URL'])
    domain_df = pd.DataFrame(domain_json)
    domain_transp = domain_df.T
    
    url_df = pd.DataFrame(url_json["pages"])
    cdomain_df = pd.DataFrame(cdomain_json)
    cdomain_transp = cdomain_df.T
    curl_df = pd.DataFrame(curl_json["pages"])
    
    if url_df.empty:
        url_df[0] = 0
    if curl_df.empty:
        curl_df[0] = 0
        
    domain_transp = domain_transp.rename(index={"domain": 0})
    cdomain_transp = cdomain_transp.rename(index={"domain": 0})

    df["DR"] = domain_transp
    df["UR"] = url_df
    df["Competitor"] = comp
    df["Comp_DR"] = cdomain_transp
    df["Comp_UR"] = curl_df
    df["Date"] = date.today()

    return df

In [ ]:
token = ""
target_url = ["target url 1"
             , "target url 2"
             ]

target_comp = ["competitor 1 url"
              , "competitor 2 url"]

#url_data = url_compare(target_url[0], target_comp[0])
for i in range(len(target_url)):
    if i == 0:
        url_data = url_compare(target_url[i], target_comp[i])
    else:
        url_data = url_data.append(url_compare(target_url[i], target_comp[i]), ignore_index=True)

url_data["Link_Points_Deficit"] = url_data["Comp_UR"]-url_data["UR"]
url_data["Required_Backlinks"] = url_data["Link_Points_Deficit"] / 0.66
display(url_data)
url_data.to_csv("data/url_data{}.csv".format(date.today()))